In [1]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,TargetEncoder

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")#uso X y Y frescas 
y = pd.read_csv("In/Training_set_labels.csv")
    
X,y = preprocess(X,y)
num_cols = X.select_dtypes(include=np.number).columns.tolist()

OrdinalEncoder_cols = X.select_dtypes(exclude=np.number).columns.tolist()
OnehotEnc_cols = []
TargetEnc_cols = []
WOEENC_cols = []# no creo que sus resultados varíen mucho del TargetEnc_cols
FrequencyEnc_cols = [] #busca category encoders

preprocessing = ColumnTransformer([
        ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value",
                                    unknown_value=10000000,
                                    encoded_missing_value =10000001,
                                    min_frequency = 0.01), OrdinalEncoder_cols),
        #arreglar:
        ('onehot', OneHotEncoder(max_categories=7,# Si existen más de 6 valores únicos crea categoría "infrequent"
                                    handle_unknown ="infrequent_if_exist",
                                    drop="first"), OnehotEnc_cols),

        ('target', TargetEncoder(target_type="multiclass",
                                    cv=2,), TargetEnc_cols),  # default cv 5 me tardaría una eternidad
                                    
        ('WOE', WOEEncoder(handle_unknown="value",# da 0
                                    handle_missing="value"), WOEENC_cols),  

        ('Frequency', CountEncoder(handle_unknown="value",
                                    handle_missing="value"), FrequencyEnc_cols),                             
        ('scaler', StandardScaler(), num_cols),  
    ],
    remainder = "passthrough",
    )

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
print("llegó")
rf = RandomForestClassifier()
rf.fit(X,y.status_group)
